# 加速：多線程爬蟲



* 了解知乎 API 使用方式與回傳內容
* 撰寫程式存取 API 且添加標頭

## 作業目標

* 找一個之前實作過的爬蟲改用多線程改寫，比較前後時間的差異。





In [1]:
import requests
import threading
import _thread
import time
import json
from bs4 import BeautifulSoup


# 爬取 即時熱門前10名看板 中各自的第一名


# 取得 "抓取 Dcard 即時熱門看板" 所需參數 pageKey
url = 'https://www.dcard.tw/service/api/v2/popularForums/GetHead?listKey=popularForums'
r = requests.get(url)
if r.status_code == requests.codes.ok:
    data = json.loads(r.text)
    page_key = data['head']
else:
    print('網頁載入失敗')
    page_key = ''

# 抓取 Dcard 即時熱門看板 前10名看板
url = f'https://www.dcard.tw/service/api/v2/popularForums/GetPage?pageKey={page_key}'
r = requests.get(url)
if r.status_code == requests.codes.ok:
    data = json.loads(r.text)
    popular_forums = data['items'][:10]
else:
    print('網頁載入失敗')
    page_key = ''


print(f'=== 即時熱門前 {len(popular_forums)} 名看板 ===')

forums_url = []
for index, popular_forum in enumerate(popular_forums):
    print(f"{index+1:2d}. {popular_forum['name']}")
    _url = f"http://dcard.tw/_api/forums/{popular_forum['alias']}/posts?popular=true"
    forums_url.append(_url)

=== 即時熱門前 10 名看板 ===
 1. 感情
 2. 心情
 3. 有趣
 4. 美妝
 5. 女孩
 6. 穿搭
 7. 追星
 8. YouTuber
 9. 美食
10. 時事


In [2]:
def get_popular_article(forum_url):
    """取得看板文章列表中的第一篇文章資訊"""
    r = requests.get(forum_url)
    if r.status_code == requests.codes.ok:
        data = json.loads(r.text)
        title = data[0]['title']
        create_tume = data[0]['createdAt']
        forum_name = data[0]['forumName']
        print(forum_name, title, create_tume)
    else:
        print(f'網頁載入失敗：{forum_url}')

In [3]:
# 沒有使用多線程
startTime = time.time()

for forum_url in forums_url:
    get_popular_article(forum_url)

finishTime = time.time()
print(f'===== "沒有"多線程 花費時間：{finishTime - startTime:.5f}秒 =====')

感情 在宿舍啪這麼大聲，驚醒（有片 2020-08-27T18:05:54.207Z
心情 被指控性騷擾 2020-08-27T15:23:50.522Z
有趣 小朋友，在補習班這樣不妥吧... 2020-08-27T15:34:55.901Z
美妝 *更* 妳一定要試試看的去除粉刺妙招! 2020-08-28T06:50:07.675Z
女孩 真的羨慕大胸嗎？ 2020-08-28T12:36:23.561Z
穿搭 ❤︎ ⌁⌁⌁⋆近期ㄉ穿搭⋆⌁⌁⌁ ❤︎ 2020-08-28T12:44:16.451Z
追星 追星板板規 2019/10/15 起生效 2019-10-15T03:46:41.000Z
YouTuber 【置頂】8月新興Youtuber精選推薦 2020-07-19T15:45:31.148Z
美食 #公告  外送優惠集中串 2020-08-20T04:13:41.507Z
時事 卡友您好，這裡是 Dcard 帳號安全小組 2019-12-26T06:19:11.620Z
===== "沒有"多線程 花費時間：2.79673秒 =====


In [4]:
# 使用 _thread 多線程
startTime = time.time()

for forum_url in forums_url:
    _thread.start_new_thread( get_popular_article, (forum_url, ) )

finishTime = time.time()
print(f'===== "使用"多線程 _thread 花費時間：{finishTime - startTime:.5f}秒 =====')

===== "使用"多線程 _thread 花費時間：0.00099秒 =====
感情 在宿舍啪這麼大聲，驚醒（有片 2020-08-27T18:05:54.207Z時事 卡友您好，這裡是 Dcard 帳號安全小組 
2019-12-26T06:19:11.620Z
YouTuber 【置頂】8月新興Youtuber精選推薦 2020-07-19T15:45:31.148Z
有趣 小朋友，在補習班這樣不妥吧... 2020-08-27T15:34:55.901Z
美妝 *更* 妳一定要試試看的去除粉刺妙招! 2020-08-28T06:50:07.675Z
女孩 真的羨慕大胸嗎？ 2020-08-28T12:36:23.561Z
心情 被指控性騷擾 2020-08-27T15:23:50.522Z
穿搭 ❤︎ ⌁⌁⌁⋆近期ㄉ穿搭⋆⌁⌁⌁ ❤︎ 2020-08-28T12:44:16.451Z
美食 #公告  外送優惠集中串 2020-08-20T04:13:41.507Z
追星 追星板板規 2019/10/15 起生效 2019-10-15T03:46:41.000Z


**但使用 _thread 套件無法取得線程的回傳值，因此以上"花費時間"是不正確的，底下改用 threading 套件實作。**

In [5]:
# 使用 threading 多線程
startTime = time.time()

job = []
for index, forum_url in enumerate(forums_url):
    job.append( threading.Thread(target=get_popular_article, args=(forum_url, )) )
    job[index].start()
    
# 等待所有子執行緒結束
for index in range(len(forums_url)):
    job[index].join()

finishTime = time.time()
print(f'===== "使用"多線程 threading 花費時間：{finishTime - startTime:.5f}秒 =====')

時事 卡友您好，這裡是 Dcard 帳號安全小組 2019-12-26T06:19:11.620Z
心情感情 在宿舍啪這麼大聲，驚醒（有片 2020-08-27T18:05:54.207Z
 被指控性騷擾 2020-08-27T15:23:50.522Z
追星 追星板板規 2019/10/15 起生效 2019-10-15T03:46:41.000Z
穿搭 ❤︎ ⌁⌁⌁⋆近期ㄉ穿搭⋆⌁⌁⌁ ❤︎ 2020-08-28T12:44:16.451Z
有趣 小朋友，在補習班這樣不妥吧... 2020-08-27T15:34:55.901Z
美食 #公告  外送優惠集中串 2020-08-20T04:13:41.507Z
美妝 *更* 妳一定要試試看的去除粉刺妙招! 2020-08-28T06:50:07.675Z
女孩 真的羨慕大胸嗎？ 2020-08-28T12:36:23.561Z
YouTuber 【置頂】8月新興Youtuber精選推薦 2020-07-19T15:45:31.148Z
===== "使用"多線程 threading 花費時間：0.49933秒 =====
